In [ ]:
from course_utils import *

# Lecture 18 – Classifier Evaluation, Model Fairness, Parting Thoughts

### Precision and recall

<center><img src="imgs/Precisionrecall.svg.png" width=30%></center>

<center>(<a href="https://en.wikipedia.org/wiki/Precision_and_recall">source</a>)</center>

<div class="alert alert-warning">
    <h3>Question 🤔 </h3>


</div>

After fitting a `BillyClassifier`, we use it to make predictions on an unseen test set. Our results are summarized in the following confusion matrix.

| | **Predicted Negative** | **Predicted Positive** |
| --- | --- | --- |
| **Actually Negative** | ??? | 30 |
| **Actually Positive** | 66 | 105 |

</div>

**Part 1**: What is the recall of our classifier? Give your answer as a fraction (it does not need to be simplified).

In [ ]:
# 105 / (105 + 66) = 105 / 171

**Part 2**: The accuracy of our classifier is $\frac{69}{117}$. How many **true negatives** did our classifier have? Give your answer as an integer.

In [ ]:
# (x + 105) / (x + 201) = 138 / 234

In [ ]:
# x = 33

**Part 3**: True or False: In order for a binary classifier's precision and recall to be equal, the number of mistakes it makes must be an even number.

In [ ]:
# TP / (TP + FP) = TP / (TP + FN)
# TP + FP = TP + FN
# FP = FN
# 
# FP + FP = 2 * FP

**Part 4**: Suppose we are building a classifier that listens to an audio source (say, from your phone’s microphone) and predicts whether or not it is Soulja Boy’s 2008 classic “Kiss Me thru the Phone." Our classifier is pretty good at detecting when the input stream is ”Kiss Me thru the Phone", but it often incorrectly predicts that similar sounding songs are also “Kiss Me thru the Phone."

Complete the sentence: Our classifier has...
- low precision and low recall.
- low precision and high recall.
- high precision and low recall.
- high precision and high recall.

## Logistic regression

### Wisconsin breast cancer dataset

The Wisconsin breast cancer dataset (WBCD) is a commonly-used dataset for demonstrating binary classification. It is built into `sklearn.datasets`.

In [ ]:
from sklearn.datasets import load_breast_cancer
loaded = load_breast_cancer() # explore the value of `loaded`!
data = loaded['data']
labels = 1 - loaded['target']
cols = loaded['feature_names']
bc = pd.DataFrame(data, columns=cols)

In [ ]:
bc.head()

1 stands for "malignant", i.e. cancerous, and 0 stands for "benign", i.e. safe.

In [ ]:
labels

In [ ]:
pd.Series(labels).value_counts(normalize=True)

Our goal is to use the features in `bc` to predict `labels`.

### Logistic regression

Logistic _regression_ is a linear _classification_ technique that builds upon linear regression. It models **the probability of belonging to class 1, given a feature vector**:

$$P(y = 1 | \vec{x}) = \sigma (\underbrace{w_0 + w_1 x^{(1)} + w_2 x^{(2)} + ... + w_d x^{(d)}}_{\text{linear regression model}})$$

Here, $\sigma(t) = \frac{1}{1 + e^{-t}}$ is the **sigmoid** function; its outputs are between 0 and 1 (which means they can be interpreted as probabilities).

🤔 **Question**: Suppose our logistic regression model predicts the probability that a tumor is malignant is 0.75. What class do we predict – malignant or benign? What if the predicted probability is 0.3?

🙋 **Answer**: We have to pick a threshold (e.g. 0.5)!
- If the predicted probability is above the threshold, we predict malignant (1).
- Otherwise, we predict benign (0).
- In practice, we use cross validation to decide this threshold.

### Fitting a logistic regression model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(bc, labels)

In [ ]:
clf = LogisticRegression(max_iter=10000)
clf.fit(X_train, y_train)

How did `clf` come up with 1s and 0s?

In [ ]:
clf.predict(X_test)

It turns out that the predicted labels come from applying a **threshold** of 0.5 to the predicted probabilities. We can access the predicted probabilities using the `predict_proba` method:

In [ ]:
# [:, 1] refers to the predicted probabilities for class 1.
clf.predict_proba(X_test)

Note that our model still has $w^*$s:

In [ ]:
clf.intercept_

In [ ]:
clf.coef_

### Evaluating our model

Let's see how well our model does on the test set.

In [ ]:
from sklearn import metrics

In [ ]:
y_pred = clf.predict(X_test)

Which metric is more important for this task – precision or recall?

In [ ]:
metrics.confusion_matrix(y_test, y_pred)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_estimator(clf, X_test, y_test);
plt.grid(False)

In [ ]:
metrics.accuracy_score(y_test, y_pred)

In [ ]:
metrics.precision_score(y_test, y_pred)

In [ ]:
metrics.recall_score(y_test, y_pred)

### What if we choose a different threshold?

🤔 **Question**: Suppose we choose a threshold **higher** than 0.5. What will happen to our model's precision and recall?

🙋 **Answer**: Precision will increase, while recall will decrease*.

- If the "bar" is higher to predict 1, then we will have fewer positives in general, and thus fewer false positives.

- The denominator in $\text{precision} = \frac{TP}{TP + FP}$ will get smaller, and so precision will increase.

- However, the number of false negatives will increase, as we are being more "strict" about what we classify as positive, and so $\text{recall} = \frac{TP}{TP + FN}$ will decrease.

- *It is possible for either or both to stay the same, if changing the threshold slightly (e.g. from 0.5 to 0.500001) doesn't change any predictions.

Similarly, if we decrease our threshold, our model's precision will decrease, while its recall will increase. 

### Trying several thresholds

The classification threshold is not actually a hyperparameter of `LogisticRegression`, because the threshold doesn't change the coefficients ($w^*$s) of the logistic regression model itself (see [this article](https://stats.stackexchange.com/questions/390186/is-decision-threshold-a-hyperparameter-in-logistic-regression#:~:text=The%20decision%20threshold%20is%20not,how%20hyper%2Dparameters%20are%20tuned.) for more details).

- Still, the threshold affects our decision rule, so we can tune it using cross-validation (which is not what we're doing below).
- It's also useful to plot how our metrics change as we change the threshold.

In [ ]:
thresholds = np.arange(0.01, 1.01, 0.01)
precisions = np.array([])
recalls = np.array([])

for t in thresholds:
    y_pred = clf.predict_proba(X_test)[:, 1] >= t
    precisions = np.append(precisions, metrics.precision_score(y_test, y_pred, zero_division=1))
    recalls = np.append(recalls, metrics.recall_score(y_test, y_pred))

Let's visualize the results.

In [ ]:
px.line(x=thresholds, y=precisions,
        labels={'x': 'Threshold', 'y': 'Precision'}, title='Precision vs. Threshold', width=600, height=400)

In [ ]:
px.line(x=thresholds, y=recalls, 
        labels={'x': 'Threshold', 'y': 'Recall'}, title='Recall vs. Threshold', width=600, height=400)

In [ ]:
px.line(x=recalls, y=precisions, hover_name=thresholds, 
        labels={'x': 'Recall', 'y': 'Precision'}, title='Precision vs. Recall')

The above curve is called a precision-recall (or PR) curve.

🤔 **Question**: Based on the PR curve above, what threshold would you choose?

### Combining precision and recall

If we care equally about a model's precision $PR$ and recall $RE$, we can combine the two using a single metric called the **F1-score**:

$$\text{F1-score} = \text{harmonic mean}(PR, RE) = 2\frac{PR \cdot RE}{PR + RE}$$

In [ ]:
pr = metrics.precision_score(y_test, clf.predict(X_test))
re = metrics.recall_score(y_test, clf.predict(X_test))

2 * pr * re / (pr + re)

In [ ]:
metrics.f1_score(y_test, clf.predict(X_test))

Both F1-score and accuracy are overall measures of a binary classifier's performance. But remember, accuracy is misleading in the presence of class imbalance, and doesn't take into account the kinds of errors the classifier makes.

In [ ]:
metrics.accuracy_score(y_test, clf.predict(X_test))

### Other evaluation metrics for binary classifiers

We just scratched the surface! This [excellent table from Wikipedia](https://en.wikipedia.org/wiki/Template:Diagnostic_testing_diagram) summarizes the many other metrics that exist.

<center><img src='imgs/wiki-table.png' width=75%></center>

If you're interested in exploring further, a good next metric to look at is **true negative rate (i.e. specificity)**, which is the analogue of recall for true negatives.

## Model fairness

### Fairness: why do we care?

- Sometimes, a model performs better for certain groups than others; in such cases we say the model is **unfair**.

- Since ML models are now used in processes that significantly affect human lives, it is important that they are fair!
    * Job applications and college admissions.
    * Criminal sentencing and parole grants.
    * Predictive policing.
    * Credit and loans.

### Model fairness

- We'd like to build a model that is _fair_, meaning that it performs the same for individuals within a group and individuals outside of the group.

- What do we mean by "perform"? What do we mean by "the same"?

<center><img src="imgs/parity.png" width=900></center>

### Parity measures for classifiers

Suppose $C$ is a classifier we've already trained, and $A$ is some binary attribute that denotes whether an individual is a member of a _sensitive_ group – that is, a group we want to avoid discrimination for (e.g. $A = \text{age is less than 25}$).

- $C$ achieves **accuracy parity** if $C$ has the same accuracy for individuals in $A$ and individuals not in $A$.
    - **Example**: $C$ is a binary classifier that determines whether someone receives a loan.
        - If the classifier predicts correctly, then either $C$ approves the loan and it is paid off, or $C$ denies the loan and it would have defaulted.
        - If $C$ achieves accuracy parity, then the proportion of correctly classified loans should be the same for those under 25 and those over 25.

- $C$ achieves **precision (or recall) parity** if $C$ has the same precision (or recall) for individuals in $A$ and individuals not in $A$.
    - Recall parity is often called "true positive rate parity."

- $C$ achieves **demographic parity** if the proportion of predictions that are positive is equal for individuals in $A$ and individuals not in $A$. 

- With the exception of demographic parity, the parity measures above all involve checking whether some evaluation metric from Lecture 17 is equal across two groups.

### More on parity measures

- Which parity metric should you care about? It depends on your specific dataset and what types of errors are important!

- Many of these parity measures are **impossible** to satisfy simultaneously!

- The classifier parity metrics mentioned on the previous slide are only a few of the many possible parity metrics. See these [fairness notes](https://afraenkel.github.io/fairness-book/content/05-parity-measures.html) for more details, including more formal explanations.

- These don't apply for regression models; for those, we may care about **RMSE parity** or **$R^2$ parity**. There is also a notion of demographic parity for regression models, but it is outside of the scope of this course.

### Example: Loan approval

As you know from Project 2, LendingClub was a "peer-to-peer lending company"; they [used to publish](https://www.lendingclub.com/info/download-data.action) a dataset describing the loans that they approved.

* `'tag'`: whether loan was repaid in full (1.0) or defaulted (0.0).
* `'loan_amnt'`: amount of the loan in dollars.
* `'emp_length'`: number of years employed.
* `'home_ownership'`: whether borrower owns (1.0) or rents (0.0).
* `'inq_last_6mths'`: number of credit inquiries in last six months.
* `'revol_bal'`: revolving balance on borrows accounts.
* `'age'`: age in years of the borrower (protected attribute).

In [ ]:
loans = pd.read_csv(Path('data') / 'loan_vars1.csv', index_col=0)
loans.head()

The total amount of money loaned was over 5 billion dollars! 

In [ ]:
loans['loan_amnt'].sum()

In [ ]:
loans.shape[0]

### Predicting `'tag'`

Let's build a classifier that predicts whether or not a loan was paid in full. If we were a bank, we could use our trained classifier to determine whether to approve someone for a loan!

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X = loans.drop('tag', axis=1)
y = loans.tag
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
clf = RandomForestClassifier(n_estimators=50)
clf.fit(X_train, y_train)

Recall, a prediction of 1 means that we predict that the loan will be paid in full.

In [ ]:
y_pred = clf.predict(X_test)
y_pred

In [ ]:
clf.score(X_test, y_test)

In [ ]:
ConfusionMatrixDisplay.from_estimator(clf, X_test, y_test);
plt.grid(False)

### Precision

$$\text{precision} = \frac{TP}{TP+FP}$$

Precision describes the **proportion of loans that were approved that would have been paid back**.

In [ ]:
metrics.precision_score(y_test, y_pred)

If we subtract the precision from 1, we get the proportion of loans that were approved that **would not** have been paid back. This is known as the **false discovery rate**.

$$\frac{FP}{TP + FP} = 1 - \text{precision}$$

In [ ]:
1 - metrics.precision_score(y_test, y_pred)

### Recall

$$\text{recall} = \frac{TP}{TP + FN}$$

Recall describes the **proportion of loans that would have been paid back that were actually approved**.

In [ ]:
metrics.recall_score(y_test, y_pred)

If we subtract the recall from 1, we get the proportion of loans that would have been paid back that **were denied**. This is known as the **false negative rate**.

$$\frac{FN}{TP + FN} = 1 - \text{recall}$$

In [ ]:
1 - metrics.recall_score(y_test, y_pred)

From both the perspective of the bank and the lendee, a high false negative rate is bad!
- The bank left money on the table – the lendee would have paid back the loan, but they weren't approved for a loan.
- The lendee deserved the loan, but weren't given one.

### False negative rate by age

In [ ]:
results = X_test
results['age_bracket'] = results['age'].apply(lambda x: 5 * (x // 5 + 1))
results['prediction'] = y_pred
results['tag'] = y_test

(
    results
    .groupby('age_bracket')
    [['tag', 'prediction']]
    .apply(lambda x: 1 - metrics.recall_score(x['tag'], x['prediction']))
    .plot(kind='bar', title='False Negative Rate by Age Group')
)

### Computing parity measures

- $C$: Our random forest classifier (1 if we approved the loan, 0 if we denied it).
- $A$: Whether or not they were under 25 (1 if under, 0 if above).

In [ ]:
results['is_young'] = (results['age'] < 25).replace({True: 'young', False: 'old'})

First, let's compute the proportion of loans that were approved in each group. If these two numbers are the same, $C$ achieves demographic parity.

In [ ]:
results.groupby('is_young')['prediction'].mean()

$C$ evidently does not achieve demographic parity – older people are approved for loans far more often! Note that this doesn't factor in whether they were _correctly_ approved or _incorrectly_ approved.

Now, let's compute the accuracy of $C$ in each group. If these two numbers are the same, $C$ achieves accuracy parity.

In [ ]:
compute_accuracy = lambda x: metrics.accuracy_score(x['tag'], x['prediction'])

In [ ]:
(
    results
    .groupby('is_young')
    [['tag', 'prediction']]
    .apply(compute_accuracy)
    .rename('accuracy')
)

Hmm... These numbers look much more similar than before!

### Is this difference in accuracy significant?

Let's run a **permutation test** to see if the difference in accuracy is significant.
- **Null Hypothesis**: The classifier's accuracy is the same for both young people and old people, and any differences are due to chance.
- **Alternative Hypothesis**: The classifier's accuracy is higher for old people.
- Test statistic: Difference in accuracy (young minus old).
- Significance level: 0.01.

In [ ]:
obs = (results
       .groupby('is_young')
       [['tag', 'prediction']]
       .apply(compute_accuracy)
       .diff()
       .iloc[-1])
obs

In [ ]:
diff_in_acc = []
for _ in range(500):
    s = (
        results[['is_young', 'prediction', 'tag']]
        .assign(is_young=np.random.permutation(results['is_young']))
        .groupby('is_young')
        [['tag', 'prediction']]
        .apply(compute_accuracy)
        .diff()
        .iloc[-1]
    )
    
    diff_in_acc.append(s)

In [ ]:
fig = pd.Series(diff_in_acc).plot(kind='hist', histnorm='probability', nbins=20,
                            title='Difference in Accuracy (Young - Old)')
fig.add_vline(x=obs, line_color='red')
fig.update_layout(xaxis_range=[-0.1, 0.05])

It seems like the difference in accuracy across the two groups **is significant**, despite being only ~5%. Thus, $C$ likely does not achieve accuracy parity.

### Ethical questions of fairness

<div class="alert alert-warning">
    <h3>Question 🤔 </h3>


</div>


- **Question**: Is it "fair" to deny loans to younger people at a higher rate?
- Make an argument for "yes", then make an argument for "no".

In [ ]:
# yes: from the bank, i want to make money
# no:  from the government, i don't want to discourage young people from potential innovations

- Federal law prevents age from being used as a determining factor in denying a loan.

Not only should we use `'age'` to determine whether or not to approve a loan, but we also shouldn't use other features that are strongly correlated with `'age'`, like `'emp_length'`.

In [ ]:
loans

## Parting Thoughts

<center><img src="imgs/ds-lifecycle.svg" width="40%"></center>

### Course goals ✅

In this course, you...

* **Practiced** translating potentially vague questions into quantitative questions about measurable observations.
* **Learned** to reason about 'black-box' processes (e.g. complicated models).
* **Understood** computational and statistical implications of working with data.
* **Learned** to use real data tools (e.g. love the documentation!).
* **Got** a taste of the "life of a data scientist".

### Course outcomes ✅

Now, you...

* Are **prepared** for internships and data science "take home" interviews!
* Are **ready** to create your own portfolio of personal projects.
* Have the **background** and **maturity** to succeed in more advanced courses.

### Topics covered ✅

We learnt a lot this quarter.

- Week 1: From BabyPandas to Pandas
- Week 2: DataFrames
- Week 3: Messy Data, Hypothesis Testing
- Week 4: Missing Values and Imputation
- Week 5: HTTP
- Week 6: Web Scraping, Regex
- Week 7: Text Features, Regression
- Week 8: Feature Engineering
- Week 9: Generalization, CV, Decision Trees
- Week 10: Random Forests, Classifier Evaluation

### Good luck on your Final Projects and beyond!